<a href="https://colab.research.google.com/github/akanksha0911/ANN-Methods/blob/main/ANN_methods_using_FAISS_Annoy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt install libomp-dev
!python -m pip install --upgrade faiss
import faiss

In [1]:
!pip install annoy

     |████████████████████████████████| 646 kB 33.1 MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp37-cp37m-linux_x86_64.whl size=391670 sha256=2d839b306182cbe4e71147cead88a6532d08429afe689f37902f6dc014b22f28
  Stored in directory: /root/.cache/pip/wheels/4f/e8/1e/7cc9ebbfa87a3b9f8ba79408d4d31831d67eea918b679a4c07
Successfully built annoy


In [2]:
import annoy

In [3]:
from annoy import AnnoyIndex
import random

In [ ]:
 !pip install sentence-transformers



In [6]:
import requests
from io import StringIO
import pandas as pd

In [7]:
res = requests.get('https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/sick2014/SICK_train.txt')
# create dataframe
data = pd.read_csv(StringIO(res.text), sep='\t')

df = pd.DataFrame(data, columns=["sentence_A"]) 
data=df
data.head()



,sentence_A
0,A group of kids is playing in a yard and an ol...
1,A group of children is playing in the house an...
2,The young boys are playing outdoors and the ma...
3,The kids are playing outdoors near a man with ...
4,The young boys are playing outdoors and the ma...


In [8]:
# we take all samples from both sentence A and B
sentences = data['sentence_A'].tolist()
sentences[:5]

['A group of kids is playing in a yard and an old man is standing in the background',
 'A group of children is playing in the house and there is no man standing in the background',
 'The young boys are playing outdoors and the man is smiling nearby',
 'The kids are playing outdoors near a man with a smile',
 'The young boys are playing outdoors and the man is smiling nearby']

In [9]:
# we take all samples from both sentence A and B
sentences = data['sentence_A'].tolist()
# sentence_b = data['sentence_B'].tolist()
# sentences.extend(sentence_b)  # merge them
len(set(sentences))  # together we have ~4.5K unique sentences

3146

In [10]:
# remove duplicates and NaN
sentences = [word for word in list(set(sentences)) if type(word) is str]

In [11]:
from sentence_transformers import SentenceTransformer
# initialize sentence transformer model
model = SentenceTransformer('bert-base-nli-mean-tokens')
# create sentence embeddings
sentence_embeddings = model.encode(sentences)
sentence_embeddings.shape

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

(3146, 768)

In [12]:
d = sentence_embeddings.shape[1]
d

768

In [13]:
nbits=d*2
index = faiss.IndexLSH(d,nbits)

In [14]:
index.is_trained

True

In [15]:
index.add(sentence_embeddings)

In [16]:
index.ntotal

3146

In [17]:
k = 4
xq = model.encode(["football"])

In [18]:

D, I = index.search(xq, k)  # search
print(I)

[[ 390 1950 2190 1682]]


In [19]:
data['sentence_A'].iloc[[2355,970,1213,2327]]

2355                     A horse is being ridden by a man
970      The woman is cracking two eggs into a glass bowl
1213    The lady is breaking eggs that are raw into a ...
2327           The woman is not putting pasta into a dish
Name: sentence_A, dtype: object

In [20]:
sentence_embeddings[I[0]]

array([[ 0.22949529,  0.21166392, -0.10311832, ...,  0.5247167 ,
        -0.51536834,  0.44770527],
       [ 0.14060092,  0.01790351,  0.0220992 , ...,  0.5507964 ,
        -0.63177884,  0.4383739 ],
       [ 0.61695975,  0.31716365, -0.16267668, ...,  0.97776896,
        -0.11566153,  0.43575767],
       [-0.0358858 ,  0.07184518, -0.12571438, ...,  0.24695744,
        -0.8649769 ,  0.34829873]], dtype=float32)

In [21]:
# and calculate the cosine similarity between each of these vectors and xq[0]
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity(sentence_embeddings[I[0]], [xq[0]])



array([[0.8367411],
       [0.7818299],
       [0.7798314],
       [0.713277 ]], dtype=float32)

In [22]:
D

array([[269., 302., 314., 368.]], dtype=float32)

**Exhaustive Search**

In [23]:
index = faiss.IndexFlatL2(d)
index.is_trained



True

In [24]:
index.add(sentence_embeddings)
index.ntotal


3146

In [25]:
k = 4
xq = model.encode(["Someone sprints with a football"])


In [26]:
D, I = index.search(xq, k)  # search
print(I)


[[ 534 1682  390  981]]


In [27]:
data['sentence_A'].iloc[[1890, 244, 1289, 227]]

1890           The girl is spraying the plants with water
244     A dog that has a brown and white coat is sitti...
1289                A girl is applying makeup to her face
227     A black helmet is being worn by a man who is p...
Name: sentence_A, dtype: object

**HNSW Implementation**

In [28]:
# set HNSW index parameters
M = 64  # number of connections each vertex will have
ef_search = 32  # depth of layers explored during search
ef_construction = 64  # depth of layers explored during index construction

# initialize index (d == 128)
index = faiss.IndexHNSWFlat(d, M)
# set efConstruction and efSearch parameters
index.hnsw.efConstruction = ef_construction
index.hnsw.efSearch = ef_search
# add data to index
index.add(sentence_embeddings)

# search as usual
D, I = index.search(sentence_embeddings[I[0]], k=5)

print(I)

[[ 534 1682  981 1950 2899]
 [1682  534 1950  981  390]
 [ 390 1950 1682 1728  534]
 [ 981  534 1682 1728  390]]


**Product Quantization**

In [29]:
nlist = 50  # how many cells
m = 8  # number of centroid IDs in final compressed vectors
bits = 8 # number of bits in each centroid

quantizer = faiss.IndexFlatL2(d)  # we keep the same L2 distance flat index
index = faiss.IndexIVFPQ(quantizer, d, nlist, m, bits) 

In [30]:
index.is_trained

False

In [31]:


index.train(sentence_embeddings)



In [32]:
index.add(sentence_embeddings)


number of nearby cells to search  with nprobe.

In [33]:
index.nprobe = 10 

In [34]:
D, I = index.search(xq, k)
print(I)


[[ 390 1682 1950  534]]


In [35]:


[f'{i}: {sentences[i]}' for i in I[0]]



['390: Two groups of people are playing football',
 '1682: A group of people playing football is running in the field',
 '1950: Different teams are playing football on a field',
 '534: A group of football players is running in the field']

**trees and graphs**

In [36]:
number_of_trees=5


index = annoy.AnnoyIndex(d)
for i, vec in enumerate(sentence_embeddings):
  index.add_item(i, vec.tolist())
index.build(number_of_trees)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  after removing the cwd from sys.path.


True

In [37]:
k=4
I = index.get_nns_by_vector(vec.tolist(), k, search_k=-1)                                         
print(I)



[3145, 2413, 2554, 70]


In [40]:
data['sentence_A'].iloc[[3145,970,1213,2327]]

3145            Four people are performing music outdoors
970      The woman is cracking two eggs into a glass bowl
1213    The lady is breaking eggs that are raw into a ...
2327           The woman is not putting pasta into a dish
Name: sentence_A, dtype: object